### Testing for Interactive use case

In [3]:
import mlflow
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from IPython.display import clear_output
import time
import platform
# from ray_on_azureml.ray_on_aml import getRay
import sys
sys.path.append("../") # go to parent dir
import importlib


In [4]:
from src.ray_on_azureml.ray_on_aml import Ray_On_AML
ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="worker-cpu-v3")
_, ray = ray_on_aml.getRay()
ray.cluster_resources()

azureml_py38
Found existing cluster, use it.


2021-12-10 03:22:40,057	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 10.0.0.5:6379


{'CPU': 48.0,
 'node:10.0.0.5': 1.0,
 'memory': 58287493940.0,
 'object_store_memory': 29143746969.0}

In [11]:
ray_on_aml.shutdown()
# import ray
# ray.shutdown()
# ray.init()

### Testing with Dask on Ray

In [7]:
# import ray
# ray.init()
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
dask.config.set(scheduler=ray_dask_get)
d_arr = da.from_array(np.random.randint(0, 1000, size=(256, 256)))

# The Dask scheduler submits the underlying task graph to Ray.
d_arr.mean().compute(scheduler=ray_dask_get)

# Set the scheduler to ray_dask_get in your config so you don't have to
# specify it on each compute call.

df = dd.from_pandas(
    pd.DataFrame(
        np.random.randint(0, 10000, size=(1024, 2)), columns=["age", "grade"]),
    npartitions=2)
df.groupby(["age"]).mean().compute()

# ray.shutdown()

,grade
age,
57,3707.0
106,4563.0
138,6669.0
171,6702.0
175,1546.0
...,...
9818,6851.0
9847,3783.0
9875,9748.0


In [1]:
import dask.dataframe as dd

storage_options = {'account_name': 'azureopendatastorage'}
ddf = dd.read_parquet('az://nyctlc/green/puYear=2019/puMonth=*/*.parquet', storage_options=storage_options)
ddf.count()

Dask Series Structure:
npartitions=1
doLocationId    int64
vendorID          ...
dtype: int64
Dask Name: dataframe-count-agg, 17 tasks

In [5]:
#dask

# import ray
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime

from azureml.core import Workspace, Dataset, Model
data_url = 'https://azureopendatastorage.blob.core.windows.net/isdweatherdatacontainer/ISDWeather'
from adlfs import AzureBlobFileSystem,AzureDatalakeFileSystem
sas_token='sp=rle&st=2021-11-27T19:26:49Z&se=2022-04-22T02:26:49Z&spr=https&sv=2020-08-04&sr=c&sig=gpYJ3LofkkjkeWkrMDTY1Ge6NsOCghbUR9CQs65HCvU%3D'
account_key='ak7nSgU/iRc5XCvXjaM2UwX18ybS9WO1BsE4Vm42XLmiZvBm1kCyZQD3USeCCOiwusWkDaq0jcRF1JpPHiAMhw=='
account_name="adlsgen7"
abfs = AzureBlobFileSystem(account_name="adlsgen7",account_key=account_key,  container_name="mltraining")
abfs2 = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")


storage_options={'account_name': account_name, 'account_key': account_key}

ddf = dd.read_parquet('az://mltraining/ISDWeatherDelta/year2008', storage_options=storage_options)

data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather/year=2009", filesystem=abfs2)
data2 = ray.data.read_parquet("az://mltraining/ISDWeatherDelta/year2008", filesystem=abfs)
data.count()

'2021.07.2'

### Testing Ray Tune for distributed ML tunning

In [9]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
# import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # In this example, we don't change the model architecture
        # due to simplicity.
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256

def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total
def train_mnist(config):
    # Data Setup
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    train_loader = DataLoader(
        datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)
    test_loader = DataLoader(
        datasets.MNIST("~/data", train=False, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)

        # Send the current training result back to Tune
        tune.report(mean_accuracy=acc)

        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")
search_space = {
    "lr": tune.sample_from(lambda spec: 10**(-10 * np.random.rand())),
    "momentum": tune.uniform(0.01, 0.09)
}

# Uncomment this to enable distributed execution
# ray.shutdown()
# ray.init(address="auto",ignore_reinit_error=True)
# ray.init(address =f'ray://{headnode_private_ip}:10001',allow_multiple=True,ignore_reinit_error=True )
# Download the dataset first
datasets.MNIST("~/data", train=True, download=True)

analysis = tune.run(train_mnist, config=search_space)


2021-12-10 03:24:14,267	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-12-10 03:24:14,281	INFO logger.py:605 -- pip install "ray[tune]" to see TensorBoard files.
2021-12-10 03:24:14,281	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,lr,momentum
train_mnist_a6475_00000,PENDING,,0.201532,0.0589635


(ImplicitFunc pid=24467) /anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
(ImplicitFunc pid=24467)   return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Result for train_mnist_a6475_00000:
  date: 2021-12-10_03-24-16
  done: false
  experiment_id: 611c4581174c40df89b7d6228c601fa2
  hostname: janguy2
  iterations_since_restore: 1
  mean_accuracy: 0.546875
  node_ip: 10.0.0.5
  pid: 24467
  time_since_restore: 1.1576545238494873
  time_this_iter_s: 1.1576545238494873
  time_total_s: 1.1576545238494873
  timestamp: 1639106656
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a6475_00000
  
Result for train_mnist_a6475_00000:
  date: 2021-12-10_03-24-17
  done: true
  experiment_id: 611c4581174c40df89b7d6228c601fa2
  experiment_tag: 0_lr=0.20153,momentum=0.058964
  hostname: janguy2
  iterations_since_restore: 10
  mean_accuracy: 0.934375
  node_ip: 10.0.0.5
  pid: 24467
  time_since_restore: 2.322474479675293
  time_this_iter_s: 0.12954235076904297
  time_total_s: 2.322474479675293
  timestamp: 1639106657
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: a6475_00000
  


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_a6475_00000,TERMINATED,10.0.0.5:24467,0.201532,0.0589635,0.934375,10,2.32247


2021-12-10 03:24:17,624	INFO tune.py:626 -- Total run time: 3.36 seconds (3.22 seconds for the tuning loop).


In [10]:
 import sklearn.datasets
 import sklearn.metrics
 from sklearn.model_selection import train_test_split
 import xgboost as xgb

 from ray import tune


 def train_breast_cancer(config):
     # Load dataset
     data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
     # Split into train and test set
     train_x, test_x, train_y, test_y = train_test_split(
         data, labels, test_size=0.25)
     # Build input matrices for XGBoost
     train_set = xgb.DMatrix(train_x, label=train_y)
     test_set = xgb.DMatrix(test_x, label=test_y)
     # Train the classifier
     results = {}
     xgb.train(
         config,
         train_set,
         evals=[(test_set, "eval")],
         evals_result=results,
         verbose_eval=False)
     # Return prediction accuracy
     accuracy = 1. - results["eval"]["error"][-1]
     tune.report(mean_accuracy=accuracy, done=True)


 config = {
     "objective": "binary:logistic",
     "eval_metric": ["logloss", "error"],
     "max_depth": tune.randint(1, 9),
     "min_child_weight": tune.choice([1, 2, 3]),
     "subsample": tune.uniform(0.5, 1.0),
     "eta": tune.loguniform(1e-4, 1e-1)
 }
 analysis = tune.run(
     train_breast_cancer,
     resources_per_trial={"cpu": 1},
     config=config,
     num_samples=10)


2021-12-10 03:24:19,734	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,min_child_weight,subsample
train_breast_cancer_a9876_00000,RUNNING,10.0.0.5:24461,0.00378889,6,3,0.802378
train_breast_cancer_a9876_00001,PENDING,,0.000491363,1,2,0.888983
train_breast_cancer_a9876_00002,PENDING,,0.00230636,1,3,0.908011
train_breast_cancer_a9876_00003,PENDING,,0.00456041,5,1,0.909479
train_breast_cancer_a9876_00004,PENDING,,0.000478117,7,2,0.735239
train_breast_cancer_a9876_00005,PENDING,,0.00181548,7,1,0.897581
train_breast_cancer_a9876_00006,PENDING,,0.000361135,5,3,0.543242
train_breast_cancer_a9876_00007,PENDING,,0.00273367,5,1,0.975624
train_breast_cancer_a9876_00008,PENDING,,0.0416131,4,1,0.95536
train_breast_cancer_a9876_00009,PENDING,,0.00919299,3,1,0.797331


Trial name,status,loc,eta,max_depth,min_child_weight,subsample
train_breast_cancer_a9876_00000,RUNNING,10.0.0.5:24461,0.00378889,6,3,0.802378
train_breast_cancer_a9876_00001,RUNNING,10.0.0.5:24462,0.000491363,1,2,0.888983
train_breast_cancer_a9876_00002,RUNNING,10.0.0.5:24479,0.00230636,1,3,0.908011
train_breast_cancer_a9876_00003,RUNNING,10.0.0.5:24466,0.00456041,5,1,0.909479
train_breast_cancer_a9876_00004,RUNNING,10.0.0.5:24469,0.000478117,7,2,0.735239
train_breast_cancer_a9876_00005,RUNNING,10.0.0.5:24471,0.00181548,7,1,0.897581
train_breast_cancer_a9876_00006,RUNNING,10.0.0.5:24407,0.000361135,5,3,0.543242
train_breast_cancer_a9876_00007,RUNNING,10.0.0.5:24470,0.00273367,5,1,0.975624
train_breast_cancer_a9876_00008,RUNNING,10.0.0.5:24463,0.0416131,4,1,0.95536
train_breast_cancer_a9876_00009,RUNNING,10.0.0.5:24472,0.00919299,3,1,0.797331


Result for train_breast_cancer_a9876_00000:
  date: 2021-12-10_03-24-28
  done: true
  experiment_id: 4b8c5fcd6681461fa96c1bef98240f5b
  hostname: janguy2
  iterations_since_restore: 1
  mean_accuracy: 0.888112
  node_ip: 10.0.0.5
  pid: 24461
  time_since_restore: 7.398552179336548
  time_this_iter_s: 7.398552179336548
  time_total_s: 7.398552179336548
  timestamp: 1639106668
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a9876_00000
  


Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_a9876_00001,RUNNING,10.0.0.5:24462,0.000491363,1,2,0.888983,,,
train_breast_cancer_a9876_00002,RUNNING,10.0.0.5:24479,0.00230636,1,3,0.908011,,,
train_breast_cancer_a9876_00003,RUNNING,10.0.0.5:24466,0.00456041,5,1,0.909479,,,
train_breast_cancer_a9876_00004,RUNNING,10.0.0.5:24469,0.000478117,7,2,0.735239,,,
train_breast_cancer_a9876_00005,RUNNING,10.0.0.5:24471,0.00181548,7,1,0.897581,,,
train_breast_cancer_a9876_00006,RUNNING,10.0.0.5:24407,0.000361135,5,3,0.543242,,,
train_breast_cancer_a9876_00007,RUNNING,10.0.0.5:24470,0.00273367,5,1,0.975624,,,
train_breast_cancer_a9876_00008,RUNNING,10.0.0.5:24463,0.0416131,4,1,0.95536,,,
train_breast_cancer_a9876_00009,RUNNING,10.0.0.5:24472,0.00919299,3,1,0.797331,,,
train_breast_cancer_a9876_00000,TERMINATED,10.0.0.5:24461,0.00378889,6,3,0.802378,0.888112,1,7.39855


...

Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_a9876_00003,RUNNING,10.0.0.5:24466,0.00456041,5,1,0.909479,,,
train_breast_cancer_a9876_00004,RUNNING,10.0.0.5:24469,0.000478117,7,2,0.735239,,,
train_breast_cancer_a9876_00005,RUNNING,10.0.0.5:24471,0.00181548,7,1,0.897581,,,
train_breast_cancer_a9876_00006,RUNNING,10.0.0.5:24407,0.000361135,5,3,0.543242,,,
train_breast_cancer_a9876_00007,RUNNING,10.0.0.5:24470,0.00273367,5,1,0.975624,,,
train_breast_cancer_a9876_00008,RUNNING,10.0.0.5:24463,0.0416131,4,1,0.95536,,,
train_breast_cancer_a9876_00009,RUNNING,10.0.0.5:24472,0.00919299,3,1,0.797331,,,
train_breast_cancer_a9876_00000,TERMINATED,10.0.0.5:24461,0.00378889,6,3,0.802378,0.888112,1,7.39855
train_breast_cancer_a9876_00001,TERMINATED,10.0.0.5:24462,0.000491363,1,2,0.888983,0.86014,1,11.0641
train_breast_cancer_a9876_00002,TERMINATED,10.0.0.5:24479,0.00230636,1,3,0.908011,0.895105,1,11.1459


Result for train_breast_cancer_a9876_00006:
  date: 2021-12-10_03-24-38
  done: true
  experiment_id: 71e3908bad894d6489608e9104300e3b
  hostname: janguy2
  iterations_since_restore: 1
  mean_accuracy: 0.93007
  node_ip: 10.0.0.5
  pid: 24407
  time_since_restore: 17.26025152206421
  time_this_iter_s: 17.26025152206421
  time_total_s: 17.26025152206421
  timestamp: 1639106678
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a9876_00006
  
Result for train_breast_cancer_a9876_00009:
  date: 2021-12-10_03-24-39
  done: true
  experiment_id: 7a7618e5041f409692686cd22eea7b3d
  hostname: janguy2
  iterations_since_restore: 1
  mean_accuracy: 0.923077
  node_ip: 10.0.0.5
  pid: 24472
  time_since_restore: 17.459272861480713
  time_this_iter_s: 17.459272861480713
  time_total_s: 17.459272861480713
  timestamp: 1639106679
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a9876_00009
  
Result for train_breast_cancer_a9876_00004:
  date: 2021-12-10_03-24-40
  done

Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_a9876_00000,TERMINATED,10.0.0.5:24461,0.00378889,6,3,0.802378,0.888112,1,7.39855
train_breast_cancer_a9876_00001,TERMINATED,10.0.0.5:24462,0.000491363,1,2,0.888983,0.86014,1,11.0641
train_breast_cancer_a9876_00002,TERMINATED,10.0.0.5:24479,0.00230636,1,3,0.908011,0.895105,1,11.1459
train_breast_cancer_a9876_00003,TERMINATED,10.0.0.5:24466,0.00456041,5,1,0.909479,0.944056,1,20.2752
train_breast_cancer_a9876_00004,TERMINATED,10.0.0.5:24469,0.000478117,7,2,0.735239,0.958042,1,19.217
train_breast_cancer_a9876_00005,TERMINATED,10.0.0.5:24471,0.00181548,7,1,0.897581,0.916084,1,20.0381
train_breast_cancer_a9876_00006,TERMINATED,10.0.0.5:24407,0.000361135,5,3,0.543242,0.93007,1,17.2603
train_breast_cancer_a9876_00007,TERMINATED,10.0.0.5:24470,0.00273367,5,1,0.975624,0.958042,1,20.2761
train_breast_cancer_a9876_00008,TERMINATED,10.0.0.5:24463,0.0416131,4,1,0.95536,0.923077,1,19.7127
train_breast_cancer_a9876_00009,TERMINATED,10.0.0.5:24472,0.00919299,3,1,0.797331,0.923077,1,17.4593


Trial name,status,loc,eta,max_depth,min_child_weight,subsample,acc,iter,total time (s)
train_breast_cancer_a9876_00003,RUNNING,10.0.0.5:24466,0.00456041,5,1,0.909479,,,
train_breast_cancer_a9876_00000,TERMINATED,10.0.0.5:24461,0.00378889,6,3,0.802378,0.888112,1,7.39855
train_breast_cancer_a9876_00001,TERMINATED,10.0.0.5:24462,0.000491363,1,2,0.888983,0.86014,1,11.0641
train_breast_cancer_a9876_00002,TERMINATED,10.0.0.5:24479,0.00230636,1,3,0.908011,0.895105,1,11.1459
train_breast_cancer_a9876_00004,TERMINATED,10.0.0.5:24469,0.000478117,7,2,0.735239,0.958042,1,19.217
train_breast_cancer_a9876_00005,TERMINATED,10.0.0.5:24471,0.00181548,7,1,0.897581,0.916084,1,20.0381
train_breast_cancer_a9876_00006,TERMINATED,10.0.0.5:24407,0.000361135,5,3,0.543242,0.93007,1,17.2603
train_breast_cancer_a9876_00007,TERMINATED,10.0.0.5:24470,0.00273367,5,1,0.975624,0.958042,1,20.2761
train_breast_cancer_a9876_00008,TERMINATED,10.0.0.5:24463,0.0416131,4,1,0.95536,0.923077,1,19.7127
train_breast_cancer_a9876_00009,TERMINATED,10.0.0.5:24472,0.00919299,3,1,0.797331,0.923077,1,17.4593


2021-12-10 03:24:42,146	INFO tune.py:626 -- Total run time: 22.42 seconds (22.29 seconds for the tuning loop).


### Testing Spark on Ray

In [1]:
import ray
import raydp
import os
ray.shutdown()
ray.init()
os.environ["PYSPARK_PYTHON"]="/anaconda/envs/azureml_py38/bin/python3"

# ray.init(address ='ray://10.0.0.11:6379')
spark = raydp.init_spark(
  app_name = "example",
  num_executors = 2,
  executor_cores = 1,
  executor_memory = "1gb"
)

# data =spark.read.format("csv").option("header", True).load("wasbs://ojsales-simulatedcontainer@azureopendatastorage.blob.core.windows.net/oj_sales_data/Store10*.csv")


# # normal data processesing with Spark
# df = spark.createDataFrame([('look',), ('spark',), ('tutorial',), ('spark',), ('look', ), ('python', )], ['word'])
# df.show()
# word_count = df.groupBy('word').count()
# word_count.show()
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declare the function and create the UDF
def multiply_func(a: pd.Series, b: pd.Series) -> pd.Series:
    return a * b

multiply = pandas_udf(multiply_func, returnType=LongType())

# The function for a pandas_udf should be able to execute with local Pandas data
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))
# 0    1
# 1    4
# 2    9
# dtype: int64

# Create a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()
# +-------------------+
# |multiply_func(x, x)|
# +-------------------+
# |                  1|
# |                  4|
# |                  9|
# +-------------------+


# stop the spark cluster
raydp.stop_spark()


2021-12-06 19:48:42,187	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8266


2021-12-06 19:48:45,043 WARN Utils [Thread-1]: Your hostname, nc6v0 resolves to a loopback address: 127.0.0.1; using 10.0.0.11 instead (on interface eth0)
2021-12-06 19:48:45,046 WARN Utils [Thread-1]: Set SPARK_LOCAL_IP if you need to bind to another address
2021-12-06 19:48:45,240 WARN NativeCodeLoader [Thread-1]: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2021-12-06 19:48:45,321 INFO SecurityManager [Thread-1]: Changing view acls to: azureuser
2021-12-06 19:48:45,321 INFO SecurityManager [Thread-1]: Changing modify acls to: azureuser
2021-12-06 19:48:45,322 INFO SecurityManager [Thread-1]: Changing view acls groups to: 
2021-12-06 19:48:45,322 INFO SecurityManager [Thread-1]: Changing modify acls groups to: 
2021-12-06 19:48:45,323 INFO SecurityManager [Thread-1]: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(azureuser); groups with view permissions: Set(); users  with modify permissions: Set(azureuser); groups with modify permissions: Set()
2021-12-06 19:48:45,591 INFO Utils [Thread-1]: Successfully started service 'RAY_RPC_ENV' on port 44621.


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/jars/ray_dist.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


2021-12-06 19:48:48,798 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-1]: Registering app example
2021-12-06 19:48:48,802 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-1]: Registered app example with ID app-20211206194848-0000
0    1
1    4
2    9
dtype: int64


(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsafe.Platform
(raylet) WARNING: Use --illegal-access=warn to enable warnings of further illegal reflective access operations
(raylet) WARNING: All illegal access operations will be denied in a future release
(raylet) WARNING: An illegal reflective access operation has occurred
(raylet) WARNING: Illegal reflective access by org.apache.spark.unsafe.Platform (file:/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/spark-unsafe_2.12-3.2.0.jar) to constructor java.nio.DirectByteBuffer(long,int)
(raylet) WARNING: Please consider reporting this to the maintainers of org.apache.spark.unsa

+-------------------+
|multiply_func(x, x)|
+-------------------+
|                  1|
|                  4|
|                  9|
+-------------------+

2021-12-06 19:48:59,086 INFO RayAppMaster [Thread-1]: Stopping RayAppMaster


In [ ]:
raydp.stop_spark()


## Testing Ray on Job Cluster

In [5]:
#     pyarrow >=6.0.1
# dask >=2021.11.2
# adlfs >=2021.10.0
# fsspec==2021.10.1
# ray[default]==1.9.0
ws = Workspace.from_config()

base_conda_dep =['adlfs>=2021.10.0','pytorch','matplotlib','torchvision','pip']
base_pip_dep = ['sklearn','xgboost','lightgbm','ray[default]==1.9.0', 'xgboost_ray', 'dask>=2021.11.2','pyarrow>=6.0.1', 'azureml-mlflow']
compute_cluster = 'worker-cpu-v3'
maxnode =5
vm_size='STANDARD_DS3_V2'
vnet='rayvnet'
subnet='default'
exp ='ray_on_aml_job'
ws_detail = ws.get_details()
ws_rg = ws_detail['id'].split("/")[4]
vnet_rg=None
try:
    ray_cluster = ComputeTarget(workspace=ws, name=compute_cluster)

    print('Found existing cluster, use it.')
except ComputeTargetException:
    if vnet_rg is None:
        vnet_rg = ws_rg
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                        min_nodes=0, max_nodes=maxnode,
                                                        vnet_resourcegroup_name=vnet_rg,
                                                        vnet_name=vnet,
                                                        subnet_name=subnet)
    ray_cluster = ComputeTarget.create(ws, compute_cluster, compute_config)

    ray_cluster.wait_for_completion(show_output=True)


python_version = ["python="+platform.python_version()]



conda_packages = python_version+base_conda_dep
pip_packages = base_pip_dep 

conda_dep = CondaDependencies()

rayEnv = Environment(name="rayEnv")
for conda_package in conda_packages:
    conda_dep.add_conda_package(conda_package)

for pip_package in pip_packages:
    conda_dep.add_pip_package(pip_package)

# Adds dependencies to PythonSection of myenv
rayEnv.python.conda_dependencies=conda_dep

src = ScriptRunConfig(source_directory='job',
                script='aml_job.py',
                environment=rayEnv,
                compute_target=ray_cluster,
                distributed_job_config=PyTorchConfiguration(node_count=maxnode),
                    # arguments = ["--master_ip",master_ip]
                )
run = Experiment(ws, exp).submit(src)

Found existing cluster, use it.
